In [301]:
import pandas as pd
import json
import requests
from pymongo import MongoClient, DESCENDING
from bson.objectid import ObjectId
import os
from pprint import pprint
import datetime as dt
import numpy as np

In [2]:
with open('D:\\lianz\\Desktop\\Python\\personal_projects\\finance_dashboard\\mongodb_api.txt','r') as f:
    cluster = f.readlines()[0]
    
client = MongoClient(cluster)

# print(client.list_database_names())

db = client.FinanceApp
balance_sheet_collection = db.balance_sheet
income_collection = db.income_statement
cash_collection = db.cash_flow_statement
company_profile = db.company_profile
historical = db.historical
stock_split = db.stock_split

In [3]:
with open('D:\\lianz\\Desktop\\Python\\personal_projects\\finance_dashboard\\fmp_api.txt','r') as f:
    fmp_api = f.readlines()[0]

with open('D:\\lianz\\Desktop\\Python\\personal_projects\\finance_dashboard\\alpha_vantage_api.txt','r') as f:
    alpha_vantage_api = f.readlines()[0]

In [ ]:
arr = os.listdir(r"D:\\lianz\Desktop\\Python\\personal_projects\\personal_finance\\balance-sheet-statement")
len(arr)

In [ ]:
def select_profile(ticker, statement):
    r = requests.get(
        f"https://financialmodelingprep.com/api/v3/{statement}/{ticker}?apikey={fmp_api}")
    r = r.json()
    return r

In [ ]:
from pymongo import ASCENDING, DESCENDING

In [ ]:
a_data = [i for i in balance_sheet_collection.find({'symbol':'AAL'}).sort('date', DESCENDING)]

In [ ]:
a_data

In [ ]:
company_profile.delete_one({})

##### Define function to access elements of an entry in database
Use function to access individual elements of a document in a collection

In [ ]:
def access_entry(collection_name, entry_name, entry_value, return_value):
    data = collection_name.find({entry_name:entry_value})

    data = [i[return_value] for i in data]

    return data

##### Get list of tickers in database

In [268]:
balance_list = list(set([i['symbol'] for i in balance_sheet_collection.find()]))
income_list = list(set([i['symbol'] for i in income_collection.find()]))
cash_list = list(set([i['symbol'] for i in cash_collection.find()]))
company_list = list(set([i['symbol'] for i in company_profile.find()]))
list_tickers = [balance_list, income_list, cash_list, company_list]
lengths = [len(i) for i in list_tickers]
missing_tickers = [i for i in list_tickers if len(i) == max(lengths)]

In [ ]:
def select_quote(ticker, statement):
    r = requests.get(
        f"https://financialmodelingprep.com/api/v3/{statement}/{ticker}?limit=120&apikey=eb29218df82acef0486b5c014ccec868")
    r = r.json()
    return r

In [184]:
terms_interested = {'Revenue': 'revenue',
                        'Gross margin%': 'grossProfitRatio',
                        'Operating Income': 'operatingIncome',
                        'Operating Margin %': 'operatingIncomeRatio',
                        'Net Income': 'netIncome',
                        'Net Income Margin': 'netIncomeRatio',
                        'Earnings per Share': 'epsdiluted',
                        'Shares Oustanding (diluted)': 'weightedAverageShsOutDil',
                        'Dividends': 'dividendsPaid',
                        'Operating Cash Flow': 'operatingCashFlow',
                        'Cap Spending': 'capitalExpenditure',
                        'Free Cash Flow': 'freeCashFlow',
                        'Free Cash Flow per Share': 'freeCashFlowpershare',
                        'Working Capital': 'totalCurrentAssets - totalCurrentLiabilities',
                        'Net Debt': 'netDebt'
                        }
statements_type = ['Income Statement', 'Cash Flow Statement', 'Balance Sheet']

In [279]:
def read_statement(type_statement, ticker):

    if type_statement == 'Income Statement':
        statement = [i for i in income_collection.find(
            {'symbol': ticker}).sort('date', DESCENDING)]

    elif type_statement == 'Cash Flow Statement':
        statement = [i for i in cash_collection.find(
            {'symbol': ticker}).sort('date', DESCENDING)]

    else:
        statement = [i for i in balance_sheet_collection.find(
            {'symbol': ticker}).sort('date', DESCENDING)]

    return statement

In [291]:
df = pd.concat([pd.DataFrame.from_records(read_statement(x, 'AAPL'), 
                                          index='calendarYear', 
                                          exclude=['_id','date','symbol','reportedCurrency','cik','fillingDate','acceptedDate','period','link','finalLink','index_id'])
                for x in statements_type], axis=1).T
df = df.loc[~df.index.duplicated(keep='first'), :].T.sort_index()

In [399]:
# retrieve latest treasury yield
def treasury(date):
    # date1 = dt.datetime.strftime(date - dt.timedelta(days=90), "%Y-%m-%d")
    # date2 = dt.datetime.strftime(date, "%Y-%m-%d")
    r = requests.get(
        f"https://financialmodelingprep.com/api/v4/treasury?from={date}&to={date}&apikey={fmp_api}"
    )
    r = r.json()
    return r

In [479]:
def select_profile(ticker, statement):
    r = requests.get(
        f"https://financialmodelingprep.com/api/v3/{statement}/{ticker}?apikey={fmp_api}")
    r = r.json()
    s = requests.get(f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={alpha_vantage_api}")
    s = s.json()

    print(r)
    print(s)
    return r,s

In [481]:
AAPL_fmp, AAPL_alpha = select_profile('AAPL', 'profile')

[{'symbol': 'AAPL', 'price': 160.25, 'beta': 1.297088, 'volAvg': 70848706, 'mktCap': 2535459439104, 'lastDiv': 0.92, 'range': '124.17-179.61', 'changes': 1.32, 'companyName': 'Apple Inc.', 'currency': 'USD', 'cik': '0000320193', 'isin': 'US0378331005', 'cusip': '037833100', 'exchange': 'NASDAQ Global Select', 'exchangeShortName': 'NASDAQ', 'industry': 'Consumer Electronics', 'website': 'https://www.apple.com', 'description': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. In addition, the company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; AirPods Max, an over-ear wireless headphone; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, and iPod touch. Further, it provides AppleCare support services; cloud services store services; and operates various 

In [494]:
len(AAPL_fmp[0].keys()) + len(AAPL_alpha.keys())

83

In [ ]:
def mergeDictionary(dict_1, dict_2):
   dict_3 = {**dict_1, **dict_2}
   for key, value in dict_3.items():
       if key in dict_1 and key in dict_2:
               dict_3[key] = [value , dict_1[key]]
   return dict_3

mergeDictionary(AAPL_fmp[0],AAPL_alpha)

In [473]:
company_profile.update_one({'symbol':'AAPL'},{'$set': AAPL_fmp[0]})

In [443]:
import math
maturity_date = dt.datetime.strptime("2027-06-30", "%Y-%m-%d")
math.trunc((maturity_date - dt.datetime.today()).days/365)

4

In [461]:
def project_metric(df, metric, past_n_years, first_n_years, second_n_years, first_growth=None, second_growth=None):
    projected = [df[metric][-1]]
    if first_growth == 0:
        avg_growth = df[metric].pct_change()[-past_n_years:].mean()
        for i in range(first_n_years+second_n_years):
            projected.append(projected[i] * (1 + avg_growth))
    else:
        for i in range(first_n_years):
            projected.append(projected[i] * (1 + first_growth))
        for i in range(first_n_years, second_n_years + first_n_years):
            projected.append(projected[i] * (1 + second_growth))
    return projected

In [466]:
revenue = 1

if revenue:
    print(revenue)
else:
    print("Revenue is None")

1


##### Intrinsic Value Calculation

In [445]:
forecast_years = 5
# Define the financials of the company
revenues = project_revenue(df, past_n_years=10, future_n_years=forecast_years)  # Revenue forecast for the next five years based on growth of past n=10 years
depreciation = df['depreciationAndAmortization'][-5:].mean()  # Depreciation as average of past n=5 years for the company
capital_expenditures = df['capitalExpenditure'][-5:].mean()  # Capital expenditures as average of past n=5 years for the company
tax_rate = (df['incomeTaxExpense'][-5:]/df['incomeBeforeTax'][-5:]).mean()  # Tax rate as average of past n=5 years for the company calculated by dividing tax expense by incomebeforetax
net_working_capital = (df['totalCurrentAssets'][-5:]-df['totalCurrentLiabilities'][-5:]).mean()  # Net working capital (current assets - current liabilities where current generally defines period of 12 months - assets that can be liquidated/debts that must be repayed within that time period)

# # Define the assumptions for standard scenario
ebitda_margin = df['ebitdaratio'][-5:].mean()  # EBITDA margin as average of past n=5 years for the company
avg_gr_choices = ['revenue','epsdiluted','dividendsPaid','netIncome']
average_growth_rate = df[avg_gr_choices[0]].pct_change()[-10:].mean() # this can be based on revenue, net income, dividendsPaid, epsdiluted, give a choice
terminal_growth_rate = min(0.05, average_growth_rate) # This limits the terminal growth rate to 5% maximum


# # Define the WACC assumptions
risk_free_rate = treasury(dt.date.today())[0]['year5']/100 # get latest 5Y treasury yield # treasury yield (2Y, 5Y, 10Y), get realtime by querying fedAPI
market_return = 0.08 # assume a 8% return is desired
profile = select_profile('AAPL','profile')[0]
beta = profile['beta'] # beta of stock
equity = profile['mktCap'] # market cap of stock
debt = df['totalDebt'][-5:].mean() # total debt of company (excluding liabilities that are not debt)

# calculate yield to maturity of company bonds
def ytm(coupon_rate, face_value, present_value, maturity_date: str):
    maturity_date = dt.datetime.strptime(maturity_date, "%Y-%m-%d")
    n_compounding_periods = math.trunc((maturity_date - dt.datetime.today()).days/365)
    num = coupon_rate + ((face_value - present_value)/n_compounding_periods)
    den = ((face_value + present_value)/2)
    YTM = num / den
    return YTM

# wacc is the minimum rate of return that the company must earn on its investments to satisfy its investors and creditors.
def wacc(risk_free_rate, beta, market_return, tax_rate, equity, debt):
    # beta of company stock
    # risk free rate using 2Y,5Y,10Y treasury yield
    # market return = annualized % return expected if investing in this stock
    cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate) # estimatino based on CAPM 
    cost_of_debt = (1 - tax_rate) * (df['interestExpense'][-5:]/df['longTermDebt'][-5:]).mean() # estimated based on weighted average of total interest expense and longterm debt
    # debt = sum of principal amounts of all outstanding debt securities issued by the company, including bonds, loans, and other debt instruments
    # equity = market cap = shares * price per share 
    total_market_value = equity + debt 
    weight_of_equity = equity/total_market_value
    weight_of_debt = debt/total_market_value
    wacc = weight_of_equity * cost_of_equity + weight_of_debt * cost_of_debt
    return wacc

# Define a function to calculate the intrinsic value
def intrinsic_value(revenues, ebitda_margin, terminal_growth_rate, wacc, tax_rate, depreciation, capex, nwc, years):
    # Calculate the free cash flows for each year
    ebitda = [revenue * ebitda_margin for revenue in revenues]
    ebit = [ebitda[i] - depreciation for i in range(len(ebitda))]
    tax_paid = [-1 * tax_rate * ebit[i] for i in range(len(ebit))]
    net_income = [ebit[i] + tax_paid[i] for i in range(len(ebit))]
    free_cash_flow = [net_income[i] - capex - nwc for i in range(len(net_income))]

    # Calculate the terminal value
    last_free_cash_flow = free_cash_flow[-1]
    terminal_value = last_free_cash_flow * (1 + terminal_growth_rate) / (wacc - terminal_growth_rate)

    # Calculate the present value of the cash flows
    discount_factors = [1 / (1 + wacc) ** i for i in range(1, years+1)]
    pv_cash_flows = [free_cash_flow[i] * discount_factors[i] for i in range(years)]
    pv_terminal_value = [terminal_value * discount_factors[-1]]
    intrinsic_value = sum(pv_cash_flows) + sum(pv_terminal_value)
    return intrinsic_value

In [450]:
intrinsic_value(revenues, ebitda_margin, terminal_growth_rate, wacc(risk_free_rate, beta, market_return, tax_rate, equity, debt), tax_rate, depreciation, capital_expenditures, net_working_capital, forecast_years)/df['weightedAverageShsOutDil'][-1]

187.8083065459422

##### Insert to database from API

##### Insert to database from API per collection

In [ ]:
# balance_sheet_collection.create_index('index_id', unique=True)
# cash_collection.create_index('index_id', unique=True)
# income_collection.create_index('index_id', unique=True)
# company_profile.create_index('index_id', unique=True)
# historical.create_index('index_id', unique=True)
stock_split.create_index('index_id', unique=True)

In [ ]:
# The web framework gets post_id from the URL and passes it as a string
def get(post_id):
    # Convert from string to ObjectId:
    document = client.db.collection.find_one({'_id': ObjectId(post_id)})

##### Delete database entries (only run when needed)

In [ ]:
balance_sheet_collection.delete_many({'symbol':'AF'})
cash_collection.delete_many({'symbol':'AF'})
income_collection.delete_many({'symbol':'AF'})

In [ ]:
balance_sheet_collection.delete_many({'symbol':'AAL','calendarYear':'2021'})
cash_collection.delete_many({'symbol':'AAL','calendarYear':'2021'})
income_collection.delete_many({'symbol':'AAL','calendarYear':'2021'})

In [212]:
aapl_hist = pd.DataFrame.from_records([x for i,x in enumerate(historical.find({'symbol':'AAPL'}))], index='date').sort_index()

In [222]:
aapl_hist.loc[aapl_hist.index.date >= dt.date(2023,1,1)].index

DatetimeIndex(['2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06',
               '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-12',
               '2023-01-13', '2023-01-17', '2023-01-18', '2023-01-19',
               '2023-01-20', '2023-01-23', '2023-01-24', '2023-01-25',
               '2023-01-26', '2023-01-27', '2023-01-30', '2023-01-31',
               '2023-02-01', '2023-02-02', '2023-02-03', '2023-02-06',
               '2023-02-07', '2023-02-08', '2023-02-09', '2023-02-10',
               '2023-02-13', '2023-02-14', '2023-02-15', '2023-02-16',
               '2023-02-17', '2023-02-21', '2023-02-22', '2023-02-23',
               '2023-02-24', '2023-02-27', '2023-02-28', '2023-03-01',
               '2023-03-02', '2023-03-03', '2023-03-07'],
              dtype='datetime64[ns]', name='date', freq=None)

In [4]:
def download_stocksplit(ticker):
    r = requests.get(
        f"https://financialmodelingprep.com/api/v3/historical-price-full/stock_split/{ticker}?apikey={fmp_api}"
    )
    r = r.json()
    return r

In [5]:
aapl_stocksplit = download_stocksplit('AAPL')

In [6]:
for i in aapl_stocksplit['historical']:
    i['index_id'] = f"{aapl_stocksplit['symbol']}_{i['date']}"
    i['date'] = dt.datetime.strptime(i['date'], '%Y-%m-%d')
    i['symbol'] = aapl_stocksplit['symbol']

In [8]:
df_stocksplit = pd.DataFrame.from_records([x for i,x in enumerate(stock_split.find({'symbol':'AAPL'}))], index='date').sort_index()

In [9]:
for i in stock_split.find({'symbol':df['symbol'][0]}):
    print(i['date'], max(df['4. close']))

2020-08-31 00:00:00 99.9900
2014-06-09 00:00:00 99.9900
2005-02-28 00:00:00 99.9900
2000-06-21 00:00:00 99.9900
1987-06-16 00:00:00 99.9900


In [10]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Candlestick(
        x=df.index[4000:], open=df[f'1. open'], high=df['2. high'], low=df['3. low'], close=df[f'4. close']),
    secondary_y=False,
)
# for i in stock_split.find({'symbol':df['symbol'][0]}):
fig.add_shape(type="line",
    x0=df_stocksplit.index[-1], x1=df_stocksplit.index[-1], y0=max(df['4. close']),
    line=dict(color="RoyalBlue",width=3)
)
# fig.add_trace(
#     go.Bar(
#         x=df.index, y=df['5. volume'], marker=dict({'color': 'darkorange'}), texttemplate="%{value:,}", textposition="inside", name="Volume"),
#     secondary_y=True,
# )


In [143]:
def stock_price_api(ticker):
    url = "https://alpha-vantage.p.rapidapi.com/query"
    headers = {"X-RapidAPI-Key": f"{alpha_vantage_api}",
               "X-RapidAPI-Host": "alpha-vantage.p.rapidapi.com"}
    querystring = {"function":"TIME_SERIES_DAILY","symbol":f"{ticker}","outputsize":"full","datatype":"json"}
    response = requests.request("GET", url=url, headers=headers, params=querystring)
    return response.json()

In [144]:
file = stock_price_api('AAPL')

In [150]:
with open("D:\\lianz\\Desktop\\Python\\personal_projects\\personal_finance\\tickers.json", mode="r") as f:
    tickers = json.load(f)

In [168]:
len(historical.distinct('symbol'))

63

##### Change price entries to double

In [177]:
historical.update_many({},
    [
    {
        '$set': {
            'open': {
                '$toDouble': '$open'
            }, 
            'high': {
                '$toDouble': '$high'
            }, 
            'low': {
                '$toDouble': '$low'
            }, 
            'close': {
                '$toDouble': '$close'
            }, 
            'volume': {
                '$toDouble': '$volume'
            }
        }
    }
])


In [ ]:
# Update date from string to datetime format
historical.update_many({},[
    {
        '$set': {
            'date': {
                '$dateFromString': {
                    'dateString': '$date', 
                    'format': '%Y-%m-%d'
                }
            }
        }
    }
])

In [171]:
# check to see if field exists
[i for i in historical.find({ 'symbol': { '$exists': False } })]

[]

In [254]:
# get CPI data
## interval=monthly and semiannual are accepted.
def get_cpi(interval):
    url = f'https://www.alphavantage.co/query?function=CPI&interval={interval}&apikey={alpha_vantage_api}'
    r = requests.get(url)
    data = r.json()

    yield data
    
for i in get_cpi('semiannual'):
    pprint(i)

{'data': [{'date': '2022-07-01', 'value': '296.963'},
          {'date': '2022-01-01', 'value': '288.347'},
          {'date': '2021-07-01', 'value': '275.703'},
          {'date': '2021-01-01', 'value': '266.236'},
          {'date': '2020-07-01', 'value': '260.065'},
          {'date': '2020-01-01', 'value': '257.557'},
          {'date': '2019-07-01', 'value': '256.903'},
          {'date': '2019-01-01', 'value': '254.412'},
          {'date': '2018-07-01', 'value': '252.125'},
          {'date': '2018-01-01', 'value': '250.089'},
          {'date': '2017-07-01', 'value': '246.163'},
          {'date': '2017-01-01', 'value': '244.076'},
          {'date': '2016-07-01', 'value': '241.237'},
          {'date': '2016-01-01', 'value': '238.778'},
          {'date': '2015-07-01', 'value': '237.769'},
          {'date': '2015-01-01', 'value': '236.265'},
          {'date': '2014-07-01', 'value': '237.088'},
          {'date': '2014-01-01', 'value': '236.384'},
          {'date': '2013-07-

In [253]:
# get real GDP data
## interval=quarterly and annual are accepted.
def get_GDP(interval):
    url = f'https://www.alphavantage.co/query?function=REAL_GDP&interval={interval}&apikey={alpha_vantage_api}'
    r = requests.get(url)
    data = r.json()

    yield data

for i in get_GDP('annual'):
    pprint(i)

{'data': [{'date': '2022-01-01', 'value': '20015.379'},
          {'date': '2021-01-01', 'value': '19609.812'},
          {'date': '2020-01-01', 'value': '18509.143'},
          {'date': '2019-01-01', 'value': '19036.052'},
          {'date': '2018-01-01', 'value': '18609.078'},
          {'date': '2017-01-01', 'value': '18076.651'},
          {'date': '2016-01-01', 'value': '17680.274'},
          {'date': '2015-01-01', 'value': '17390.295'},
          {'date': '2014-01-01', 'value': '16932.051'},
          {'date': '2013-01-01', 'value': '16553.348'},
          {'date': '2012-01-01', 'value': '16253.97'},
          {'date': '2011-01-01', 'value': '15891.534'},
          {'date': '2010-01-01', 'value': '15648.991'},
          {'date': '2009-01-01', 'value': '15236.262'},
          {'date': '2008-01-01', 'value': '15642.962'},
          {'date': '2007-01-01', 'value': '15623.871'},
          {'date': '2006-01-01', 'value': '15315.943'},
          {'date': '2005-01-01', 'value': '14901.

In [255]:
# get treasury yield
## interval=daily, weekly, and monthly are accepted
## maturity=3month, 2year, 5year, 7year, 10year, and 30year are accepted
def get_treasury(interval, maturity):
    url = f'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval={interval}&maturity={maturity}&apikey={alpha_vantage_api}'
    r = requests.get(url)
    data = r.json()

    yield data

for i in get_treasury('monthly','2year'):
    pprint(i)

{'data': [{'date': '2023-02-01', 'value': '4.53'},
          {'date': '2023-01-01', 'value': '4.21'},
          {'date': '2022-12-01', 'value': '4.29'},
          {'date': '2022-11-01', 'value': '4.50'},
          {'date': '2022-10-01', 'value': '4.38'},
          {'date': '2022-09-01', 'value': '3.86'},
          {'date': '2022-08-01', 'value': '3.25'},
          {'date': '2022-07-01', 'value': '3.04'},
          {'date': '2022-06-01', 'value': '3.00'},
          {'date': '2022-05-01', 'value': '2.62'},
          {'date': '2022-04-01', 'value': '2.54'},
          {'date': '2022-03-01', 'value': '1.91'},
          {'date': '2022-02-01', 'value': '1.44'},
          {'date': '2022-01-01', 'value': '0.98'},
          {'date': '2021-12-01', 'value': '0.68'},
          {'date': '2021-11-01', 'value': '0.51'},
          {'date': '2021-10-01', 'value': '0.39'},
          {'date': '2021-09-01', 'value': '0.24'},
          {'date': '2021-08-01', 'value': '0.22'},
          {'date': '2021-07-01'

In [252]:
# get federal funds rate
## interval=daily, weekly, and monthly are accepted.
def get_fed_rate(interval):
    url = f'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval={interval}&apikey={alpha_vantage_api}'
    r = requests.get(url)
    data = r.json()

    yield data

for i in get_fed_rate('monthly'):
    pprint(i)

{'data': [{'date': '2023-02-01', 'value': '4.57'},
          {'date': '2023-01-01', 'value': '4.33'},
          {'date': '2022-12-01', 'value': '4.10'},
          {'date': '2022-11-01', 'value': '3.78'},
          {'date': '2022-10-01', 'value': '3.08'},
          {'date': '2022-09-01', 'value': '2.56'},
          {'date': '2022-08-01', 'value': '2.33'},
          {'date': '2022-07-01', 'value': '1.68'},
          {'date': '2022-06-01', 'value': '1.21'},
          {'date': '2022-05-01', 'value': '0.77'},
          {'date': '2022-04-01', 'value': '0.33'},
          {'date': '2022-03-01', 'value': '0.20'},
          {'date': '2022-02-01', 'value': '0.08'},
          {'date': '2022-01-01', 'value': '0.08'},
          {'date': '2021-12-01', 'value': '0.08'},
          {'date': '2021-11-01', 'value': '0.08'},
          {'date': '2021-10-01', 'value': '0.08'},
          {'date': '2021-09-01', 'value': '0.08'},
          {'date': '2021-08-01', 'value': '0.09'},
          {'date': '2021-07-01'

In [256]:
# get retail sales data

def get_retail_sales():
    url = f'https://www.alphavantage.co/query?function=RETAIL_SALES&apikey={alpha_vantage_api}'
    r = requests.get(url)
    data = r.json()

    yield data

for i in get_retail_sales():
    pprint(i)

{'data': [{'date': '2023-01-01', 'value': '540735'},
          {'date': '2022-12-01', 'value': '657682'},
          {'date': '2022-11-01', 'value': '609862'},
          {'date': '2022-10-01', 'value': '597349'},
          {'date': '2022-09-01', 'value': '577370'},
          {'date': '2022-08-01', 'value': '613416'},
          {'date': '2022-07-01', 'value': '600746'},
          {'date': '2022-06-01', 'value': '609933'},
          {'date': '2022-05-01', 'value': '614474'},
          {'date': '2022-04-01', 'value': '594426'},
          {'date': '2022-03-01', 'value': '597077'},
          {'date': '2022-02-01', 'value': '506400'},
          {'date': '2022-01-01', 'value': '518146'},
          {'date': '2021-12-01', 'value': '632849'},
          {'date': '2021-11-01', 'value': '579687'},
          {'date': '2021-10-01', 'value': '557737'},
          {'date': '2021-09-01', 'value': '532840'},
          {'date': '2021-08-01', 'value': '553822'},
          {'date': '2021-07-01', 'value': '554

In [257]:
# get durables data
def get_durables():
    url = 'https://www.alphavantage.co/query?function=DURABLES&apikey=demo'
    r = requests.get(url)
    data = r.json()

    yield data
    
for i in get_durables():
    pprint(i)

{'data': [{'date': '2023-01-01', 'value': '250074'},
          {'date': '2022-12-01', 'value': '296687'},
          {'date': '2022-11-01', 'value': '259606'},
          {'date': '2022-10-01', 'value': '273120'},
          {'date': '2022-09-01', 'value': '289932'},
          {'date': '2022-08-01', 'value': '279323'},
          {'date': '2022-07-01', 'value': '249456'},
          {'date': '2022-06-01', 'value': '294470'},
          {'date': '2022-05-01', 'value': '263971'},
          {'date': '2022-04-01', 'value': '260366'},
          {'date': '2022-03-01', 'value': '294088'},
          {'date': '2022-02-01', 'value': '250105'},
          {'date': '2022-01-01', 'value': '242492'},
          {'date': '2021-12-01', 'value': '267477'},
          {'date': '2021-11-01', 'value': '244327'},
          {'date': '2021-10-01', 'value': '247086'},
          {'date': '2021-09-01', 'value': '260121'},
          {'date': '2021-08-01', 'value': '250971'},
          {'date': '2021-07-01', 'value': '228

In [259]:
def get_unemployment():
    url = f'https://www.alphavantage.co/query?function=UNEMPLOYMENT&apikey={alpha_vantage_api}'
    r = requests.get(url)
    data = r.json()

    yield data

for i in get_unemployment():
    pprint(i)

{'data': [{'date': '2023-01-01', 'value': '3.4'},
          {'date': '2022-12-01', 'value': '3.5'},
          {'date': '2022-11-01', 'value': '3.6'},
          {'date': '2022-10-01', 'value': '3.7'},
          {'date': '2022-09-01', 'value': '3.5'},
          {'date': '2022-08-01', 'value': '3.7'},
          {'date': '2022-07-01', 'value': '3.5'},
          {'date': '2022-06-01', 'value': '3.6'},
          {'date': '2022-05-01', 'value': '3.6'},
          {'date': '2022-04-01', 'value': '3.6'},
          {'date': '2022-03-01', 'value': '3.6'},
          {'date': '2022-02-01', 'value': '3.8'},
          {'date': '2022-01-01', 'value': '4.0'},
          {'date': '2021-12-01', 'value': '3.9'},
          {'date': '2021-11-01', 'value': '4.2'},
          {'date': '2021-10-01', 'value': '4.5'},
          {'date': '2021-09-01', 'value': '4.8'},
          {'date': '2021-08-01', 'value': '5.2'},
          {'date': '2021-07-01', 'value': '5.4'},
          {'date': '2021-06-01', 'value': '5.9'},


In [261]:
def get_nonfarm_payroll():
    url = f'https://www.alphavantage.co/query?function=NONFARM_PAYROLL&apikey={alpha_vantage_api}'
    r = requests.get(url)
    data = r.json()

    yield data

for i in get_nonfarm_payroll():
    pprint(i)

{'data': [{'date': '2023-01-01', 'value': '152844'},
          {'date': '2022-12-01', 'value': '155349'},
          {'date': '2022-11-01', 'value': '155642'},
          {'date': '2022-10-01', 'value': '155041'},
          {'date': '2022-09-01', 'value': '153809'},
          {'date': '2022-08-01', 'value': '153285'},
          {'date': '2022-07-01', 'value': '152875'},
          {'date': '2022-06-01', 'value': '153217'},
          {'date': '2022-05-01', 'value': '152291'},
          {'date': '2022-04-01', 'value': '151449'},
          {'date': '2022-03-01', 'value': '150411'},
          {'date': '2022-02-01', 'value': '149606'},
          {'date': '2022-01-01', 'value': '147932'},
          {'date': '2021-12-01', 'value': '150740'},
          {'date': '2021-11-01', 'value': '150543'},
          {'date': '2021-10-01', 'value': '149605'},
          {'date': '2021-09-01', 'value': '147917'},
          {'date': '2021-08-01', 'value': '147159'},
          {'date': '2021-07-01', 'value': '146

In [477]:
def stock_peers(ticker):
    r = requests.get(
        f"https://financialmodelingprep.com/api/v4/stock_peers?symbol={ticker}&apikey={fmp_api}"
    )
    r = r.json()
    return r